In [1]:
import os
import sys
import re
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import copy
import pickle
from scipy.optimize import fmin
import multiprocessing as mp
import functools

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
# statmech library setup
sys.path.append('../../statmechlib')
from statmechlib.preprocessing import Trajectory, select_nodes, scale_configuration, pair_dist_cutoff
from statmechlib.forcefields import sd2_loss, utot_EAM_per_atom, utot_EAM_per_box, ftot_EAM, udif_print, u_core
from statmechlib.preprocessing import universal_eos
from statmechlib.preprocessing import pair_dist, force_targ, get_stats_EAM_per_atom, get_stats_EAM_per_box

In [4]:
target_raw = '../data/target_raw'
target_proc = '../data/target_processed'
working = '../data/working'

In [5]:
with open(os.path.join(working, 'stats_samples'+'.pickle'), 'rb') as fi:
    stats_all = pickle.load(fi)

with open(os.path.join(working, 'target_samples'+'.pickle'), 'rb') as fi:
    targets = pickle.load(fi)   

In [6]:
# show all available spline knots
stats_all['hyperparams']

{'pair': [2.74, 3.252, 3.804, 4.356, 4.908, 5.46],
 'embed': [2.74, 3.252, 3.804, 4.356, 4.908, 5.46]}

In [7]:
stats_all['hyperparams']['edens'] = [2.74, 3.252, 3.804, 4.356, 4.908, 5.46]
del stats_all['hyperparams']['embed']

In [8]:
# selecty spline knots
pair_index = [0, 1, 2, 3, 4, 5]
multibody_index = [2, 5]

p_ix = np.array([True if i in pair_index else False for i in range(len(stats_all['hyperparams']['pair']))])
m_ix = np.array([True if i in multibody_index else False for i in range(len(stats_all['hyperparams']['edens']))])
print("pair:", np.array(stats_all['hyperparams']['pair'])[p_ix])
print("multibody:", np.array(stats_all['hyperparams']['edens'])[m_ix])

stats = select_nodes(stats_all, p_ix, m_ix)

pair: [2.74  3.252 3.804 4.356 4.908 5.46 ]
multibody: [3.804 5.46 ]


In [9]:
# read corresponding parameters
with open(os.path.join(working, 'pars_in'+'.pickle'), 'rb') as fi:
    pars_list = pickle.load(fi)
    
pars_dict = pars_list[0]
pars_in = pars_list[0]['embed'] + pars_list[0]['pair'] + [0.0]
print(pars_list)
print(pars_in)

[{'embed': [-0.684229349, 0.000152487644], 'pair': [-65.6958142, 23.0194161, -3.48639263, 3.45615281, -0.88808358, -1.34315802]}]
[-0.684229349, 0.000152487644, -65.6958142, 23.0194161, -3.48639263, 3.45615281, -0.88808358, -1.34315802, 0.0]


In [10]:
multi_pars = []

#multi_pars.append(pars_in)

multi_pars.append(np.array([-6.25140612e-01,  5.84924686e-05,  1.50463356e+01, -2.45628080e+00,
        2.75467959e+00, -1.82521768e+00,  5.82623873e-01, -8.10349686e-02, 0.0]))

#multi_pars.append([-1.48107607e+00, -2.11281591e-06, -7.75748552e+01,  1.23284902e+02,
#       -1.05876828e+02,  2.56371829e+01,  7.21885424e-01, -2.01549825e+00, 0.0])

#multi_pars.append([ 4.21813176e+00,  1.28438711e-02,  7.18447209e+02,  2.79873504e+01,
#       -3.64063120e+01,  1.55668343e+01, -3.59671207e+00,  7.18079901e-02, 0.0])

#multi_pars.append([-1.70284536e+00,  1.77584742e-03, -2.03091714e+02,  6.67172987e+02,
#       -1.39072153e+03,  7.95275451e+02, -2.44611978e+01,  5.09001525e+00, 0.0])

#multi_pars.append([ 1.49705729e+00,  8.14531512e-03, -8.15988329e+02,  5.87519886e+02,
#       -1.01285873e+03,  5.10026120e+02, -1.11686150e+01,  1.51343754e+00, 0.0])

#multi_pars.append([ 1.54864560e+00,  1.06062750e-02, -3.00996729e+02,  3.32437763e+02,
#       -3.39714171e+02,  9.93126859e+01, -2.04873166e+01,  1.12932344e+00, 0.0])

#multi_pars.append([ 1.08153442e+00,  2.12340093e-03, -2.83828424e+02,  3.30196594e+02,
#       -3.61260411e+02,  1.30278313e+02, -3.12845720e+01,  2.97822976e+00, 0.0])

#multi_pars.append([-1.84530425e+00, -2.18389467e-05, -1.46520729e+03,  1.16790147e+02,
#       -1.41528209e+02,  7.82448301e+01,  4.04340173e+00, -2.96638562e+00, 0.0])

In [11]:
targets['md']['weight'] = 1.0
targets['relax']['weight'] = 20.0
targets['eos_bcc']['weight'] = 1.0
targets['eos_fcc']['weight'] = 0.1

In [12]:
def optimize_EAM_box_mp(pars, targets, stats, n_b=3):
    
    output = fmin(sd2_loss, pars, args=(targets, stats, utot_EAM_per_box), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)
    
    return tuple([output[1], output[0]])

In [13]:
def optimize_EAM_mp(pars, targets, stats, n_b=3):
    
    output = fmin(sd2_loss, pars, args=(targets, stats, utot_EAM_per_atom, None, 0.05, 1), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)
        
    return tuple([output[1], output[0]])

In [14]:
pool = mp.Pool()

for ieam in range(1):#5, len(stats_opts[it]['hyperparams'])):

    get_opt = functools.partial(optimize_EAM_mp, targets=targets, stats=stats, n_b=len(multi_pars))

    get_sd = functools.partial(sd2_loss, targets=targets, stats=stats, utot_func=utot_EAM_per_atom, ftot_func=None, dl=0.05, verbose=1)
  
    # initial ordering
    print('Initial params:', multi_pars)
    
    distances = pool.map(get_sd, multi_pars)
    
    optimal_params = zip(distances, multi_pars)
    best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
    m_pars = [p[1] for p in best_params]

    print('Best params:', best_params)
    print('ieam:', ieam)
    #m_pars = multi_pars

    for i in range(6):
        optimal_params = pool.map(get_opt, m_pars)
        best_params = sorted(optimal_params, key=lambda param: param[0], reverse=True)
        m_pars = [p[1] for p in best_params]
        
        print(f"Iteration {i}, best params: {best_params}")

    print('Final Best params:', best_params)

pool.close()
pool.join()

Initial params: [array([-6.25140612e-01,  5.84924686e-05,  1.50463356e+01, -2.45628080e+00,
        2.75467959e+00, -1.82521768e+00,  5.82623873e-01, -8.10349686e-02,
        0.00000000e+00])]
loss 0.0049266800892666 0.0049266800892666 0.0
Best params: [(0.0049266800892666, array([-6.25140612e-01,  5.84924686e-05,  1.50463356e+01, -2.45628080e+00,
        2.75467959e+00, -1.82521768e+00,  5.82623873e-01, -8.10349686e-02,
        0.00000000e+00]))]
ieam: 0
loss 0.0049266800892666 0.0049266800892666 0.0
loss 0.2089153466603414 0.2089153466603414 0.0
loss 0.04432178447004907 0.04432178447004907 0.0
loss 0.005585764731018873 0.005585764731018873 0.0
loss 0.014889485586207866 0.014889485586207866 0.0
loss 0.8395317629639862 0.8395317629639862 0.0
loss 2.3237929488857403 2.3237929488857403 0.0
loss 4.499504380011762 4.499504380011762 0.0
loss 0.2268432517972866 0.2268432517972866 0.0
loss 0.004926669051959105 0.004926669051959105 0.0
loss 2.2524416274679857 2.2524416274679857 0.0
loss 3.5743

loss 0.004927618650836683 0.004927618650836683 0.0
loss 0.004926856802527022 0.004926856802527022 0.0
loss 0.0049276213691662185 0.0049276213691662185 0.0
loss 0.004926819652098242 0.004926819652098242 0.0
loss 0.004927385875834025 0.004927385875834025 0.0
loss 0.004926837769247666 0.004926837769247666 0.0
loss 0.0049269208249102065 0.0049269208249102065 0.0
loss 0.004927011129841443 0.004927011129841443 0.0
loss 0.004926753307075306 0.004926753307075306 0.0
loss 0.0049271407858692585 0.0049271407858692585 0.0
loss 0.004926734315513253 0.004926734315513253 0.0
loss 0.004927022117404367 0.004927022117404367 0.0
loss 0.004926735817979101 0.004926735817979101 0.0
loss 0.004927022830270796 0.004927022830270796 0.0
loss 0.0049267318521191514 0.0049267318521191514 0.0
loss 0.00492686776859179 0.00492686776859179 0.0
loss 0.0049267250986482925 0.0049267250986482925 0.0
loss 0.004926822908518369 0.004926822908518369 0.0
loss 0.004926833935580737 0.004926833935580737 0.0
loss 0.0049267148216774

loss 0.004925849816567597 0.004925849816567597 0.0
loss 0.0049258667627766275 0.0049258667627766275 0.0
loss 0.004925820209941872 0.004925820209941872 0.0
loss 0.004925812362592494 0.004925812362592494 0.0
loss 0.004925764922108167 0.004925764922108167 0.0
loss 0.004925787957623594 0.004925787957623594 0.0
loss 0.004925757939322761 0.004925757939322761 0.0
loss 0.004925885311543227 0.004925885311543227 0.0
loss 0.004925778483494696 0.004925778483494696 0.0
loss 0.004925771049719047 0.004925771049719047 0.0
loss 0.00492571984812314 0.00492571984812314 0.0
loss 0.004925732432422364 0.004925732432422364 0.0
loss 0.004925759578386953 0.004925759578386953 0.0
loss 0.0049256836052053235 0.0049256836052053235 0.0
loss 0.00492566459988727 0.00492566459988727 0.0
loss 0.004925719358914761 0.004925719358914761 0.0
loss 0.004925725960710244 0.004925725960710244 0.0
loss 0.004925736448043343 0.004925736448043343 0.0
loss 0.004925766023564444 0.004925766023564444 0.0
loss 0.004925715721726301 0.004

loss 0.004923503811304544 0.004923503811304544 0.0
loss 0.004923552057930717 0.004923552057930717 0.0
loss 0.004923505574376106 0.004923505574376106 0.0
loss 0.004923581700603779 0.004923581700603779 0.0
loss 0.004923502959260024 0.004923502959260024 0.0
loss 0.004923548007063542 0.004923548007063542 0.0
loss 0.00492350544077964 0.00492350544077964 0.0
loss 0.004923513529062073 0.004923513529062073 0.0
loss 0.004923536397130551 0.004923536397130551 0.0
loss 0.0049235026791825814 0.0049235026791825814 0.0
loss 0.004923569938219119 0.004923569938219119 0.0
loss 0.004923499202660827 0.004923499202660827 0.0
loss 0.004923537071263955 0.004923537071263955 0.0
loss 0.004923502735588886 0.004923502735588886 0.0
loss 0.004923497048917893 0.004923497048917893 0.0
loss 0.004923526806901595 0.004923526806901595 0.0
loss 0.004923501285279683 0.004923501285279683 0.0
loss 0.004923510046861045 0.004923510046861045 0.0
loss 0.004923496313012056 0.004923496313012056 0.0
loss 0.0049234944518471374 0.00

loss 0.004920179737032122 0.004920179737032122 0.0
loss 0.004920504079058871 0.004920504079058871 0.0
loss 0.0049203248024958235 0.0049203248024958235 0.0
loss 0.004920385912539037 0.004920385912539037 0.0
loss 0.004920180830321829 0.004920180830321829 0.0
loss 0.004920944523142436 0.004920944523142436 0.0
loss 0.004920385577244799 0.004920385577244799 0.0
loss 0.00492002858753089 0.00492002858753089 0.0
loss 0.004920132681093207 0.004920132681093207 0.0
loss 0.004919802770893158 0.004919802770893158 0.0
loss 0.004919623712895832 0.004919623712895832 0.0
loss 0.004919913126833528 0.004919913126833528 0.0
loss 0.004919594669896794 0.004919594669896794 0.0
loss 0.004919465333106216 0.004919465333106216 0.0
loss 0.004920522763840711 0.004920522763840711 0.0
loss 0.0049199586069239 0.0049199586069239 0.0
loss 0.004919443182364853 0.004919443182364853 0.0
loss 0.004919278886106506 0.004919278886106506 0.0
loss 0.004919871531401686 0.004919871531401686 0.0
loss 0.004919739924430801 0.0049197

...

loss 0.005093874562819939 0.005093874562819939 0.0
loss 0.005118099714349461 0.005118099714349461 0.0
loss 0.00520079727174266 0.00520079727174266 0.0
loss 0.004921997060637217 0.004921997060637217 0.0
loss 0.005323342587049969 0.005323342587049969 0.0
loss 0.0049080933503449385 0.0049080933503449385 0.0
loss 0.0053518714772497495 0.0053518714772497495 0.0
loss 0.00490612990131244 0.00490612990131244 0.0
loss 0.00507347418832031 0.00507347418832031 0.0
loss 0.005114767056413134 0.005114767056413134 0.0
loss 0.004912521696871159 0.004912521696871159 0.0
loss 0.005118848142526645 0.005118848142526645 0.0
loss 0.0049073989488896096 0.0049073989488896096 0.0
loss 0.0050691934347353255 0.0050691934347353255 0.0
loss 0.004886859025188418 0.004886859025188418 0.0
loss 0.005000627133187333 0.005000627133187333 0.0
loss 0.0048802368354604305 0.0048802368354604305 0.0
loss 0.005046130283088732 0.005046130283088732 0.0
loss 0.004875267549509183 0.004875267549509183 0.0
loss 0.004930241804029459 0

loss 0.004856078418587394 0.004856078418587394 0.0
loss 0.004856081595158378 0.004856081595158378 0.0
loss 0.004856078328057237 0.004856078328057237 0.0
loss 0.00485607858516549 0.00485607858516549 0.0
loss 0.004856078203150624 0.004856078203150624 0.0
loss 0.004856079128535038 0.004856079128535038 0.0
loss 0.004856077806029641 0.004856077806029641 0.0
loss 0.0048560782100036184 0.0048560782100036184 0.0
loss 0.004856079288249984 0.004856079288249984 0.0
loss 0.004856077697775887 0.004856077697775887 0.0
loss 0.00485607901883801 0.00485607901883801 0.0
loss 0.004856077695481772 0.004856077695481772 0.0
loss 0.004856079228017857 0.004856079228017857 0.0
loss 0.004856077637145515 0.004856077637145515 0.0
loss 0.004856078533401463 0.004856078533401463 0.0
loss 0.004856077691533194 0.004856077691533194 0.0
loss 0.0048560782038365585 0.0048560782038365585 0.0
loss 0.00485607775993571 0.00485607775993571 0.0
loss 0.004856078894903335 0.004856078894903335 0.0
loss 0.004856077539450738 0.00485

loss 0.171445238291118 0.171445238291118 0.0
loss 0.030674052822669585 0.030674052822669585 0.0
loss 0.18568197859258853 0.18568197859258853 0.0
loss 0.02425277873869577 0.02425277873869577 0.0
loss 0.078977665773105 0.078977665773105 0.0
loss 0.02091062302317124 0.02091062302317124 0.0
loss 0.12734004027389162 0.12734004027389162 0.0
loss 0.019743347209673665 0.019743347209673665 0.0
loss 0.08818651728948694 0.08818651728948694 0.0
loss 0.011580123335067166 0.011580123335067166 0.0
loss 0.0211440942955812 0.0211440942955812 0.0
loss 0.032843470735225276 0.032843470735225276 0.0
loss 0.008915614187383547 0.008915614187383547 0.0
loss 0.038159639180592377 0.038159639180592377 0.0
loss 0.008500838626487803 0.008500838626487803 0.0
loss 0.020607589031934583 0.020607589031934583 0.0
loss 0.022097354958724738 0.022097354958724738 0.0
loss 0.008660384808569122 0.008660384808569122 0.0
loss 0.024836548798216743 0.024836548798216743 0.0
loss 0.008359594337143467 0.008359594337143467 0.0
loss 0

loss 0.004856211271997217 0.004856211271997217 0.0
loss 0.0048561185561205086 0.0048561185561205086 0.0
loss 0.004856238232428423 0.004856238232428423 0.0
loss 0.0048561169090961635 0.0048561169090961635 0.0
loss 0.004856159396691985 0.004856159396691985 0.0
loss 0.004856331427562978 0.004856331427562978 0.0
loss 0.004856093838536542 0.004856093838536542 0.0
loss 0.004856161832229613 0.004856161832229613 0.0
loss 0.004856106090129651 0.004856106090129651 0.0
loss 0.004856187477314505 0.004856187477314505 0.0
loss 0.004856103048384781 0.004856103048384781 0.0
loss 0.004856116295678682 0.004856116295678682 0.0
loss 0.004856142760883111 0.004856142760883111 0.0
loss 0.004856096100242968 0.004856096100242968 0.0
loss 0.0048561200834236935 0.0048561200834236935 0.0
loss 0.00485613645306209 0.00485613645306209 0.0
loss 0.004856090484321191 0.004856090484321191 0.0
loss 0.004856134051216227 0.004856134051216227 0.0
loss 0.004856089268906405 0.004856089268906405 0.0
loss 0.004856098772993363 0

loss 0.0048560772531640916 0.0048560772531640916 0.0
loss 0.004856077251425507 0.004856077251425507 0.0
loss 0.00485607725313131 0.00485607725313131 0.0
loss 0.004856077251340144 0.004856077251340144 0.0
loss 0.004856077251936528 0.004856077251936528 0.0
loss 0.004856077251779818 0.004856077251779818 0.0
loss 0.0048560772524904264 0.0048560772524904264 0.0
loss 0.004856077251155358 0.004856077251155358 0.0
loss 0.004856077252510846 0.004856077252510846 0.0
loss 0.004856077251119705 0.004856077251119705 0.0
loss 0.004856077251969166 0.004856077251969166 0.0
loss 0.004856077251155116 0.004856077251155116 0.0
loss 0.0048560772519129275 0.0048560772519129275 0.0
loss 0.004856077251120687 0.004856077251120687 0.0
loss 0.004856077251287216 0.004856077251287216 0.0
loss 0.004856077251418081 0.004856077251418081 0.0
loss 0.004856077250667562 0.004856077250667562 0.0
loss 0.004856077251256448 0.004856077251256448 0.0
loss 0.004856077251236763 0.004856077251236763 0.0
loss 0.004856077250811691 0

In [15]:
def optimize_EAM_per_atom(targets, stats, ieam, multi_pars, n_b=3):
    
    optimal_parameters = []
    
    for i, pars in enumerate(multi_pars):
        sd_ini = sd2_loss(list(pars), stats, targets, utot_EAM_per_box, None, [ieam])
        print(i,' sd2: ini', sd_ini, end='')
        output = fmin(sd2_loss, pars, args=(stats, targets, utot_EAM_per_box, None, [ieam]), maxiter=100000, maxfun=100000, disp=0, full_output=1,ftol=1e-6)
        optimal_parameters.append(tuple([output[1], output[0]]))
        print('sd2: final', output[1])

    n_best = min(n_b, len(optimal_parameters))
    #print(optimal_parameters)
                 
    best_params = sorted(optimal_parameters, key=lambda param: param[0])[:n_best]
    
    print('Best params:', best_params)

    return best_params

In [16]:
for ieam in range(4, len(stats_opts[it]['hyperparams'])):
    print('ieam:', ieam)
    m_pars = multi_pars
    for i in range(6):
        best_pars = optimize_EAM(targets, stats, ieam, m_pars, n_b=len(m_pars))
        m_pars = [p[1] for p in best_pars]

NameError: name 'stats_opts' is not defined

In [ ]:
optimal_parameters = [(4.407570269501649e-05, [-1.21658318e+00,  3.24863580e-05, -4.73661687e+01,  2.29965951e+01,
       -3.42248343e+00,  3.73956980e+00, -1.39292737e+00, -8.33458565e-01,
        9.29769431e-01, -1.83568989e-01]), (6.17447812420396e-05, [-1.30551487e+00,  1.41920657e-05, -4.61160967e+01,  2.29352328e+01,
       -3.42360958e+00,  3.82313395e+00, -1.54715703e+00, -6.97925842e-01,
        8.68730239e-01, -1.54592379e-01]), (9.723561709685708e-05, [-1.34624502e+00,  7.49457487e-06, -4.58871674e+01,  2.24064873e+01,
       -3.46516902e+00,  4.02263760e+00, -1.89661680e+00, -4.31617423e-01,
        8.06406324e-01, -1.37562101e-01]), (0.00014123622250142564, [-1.34136558e+00,  1.01769934e-05, -4.93998032e+01,  2.21279115e+01,
       -3.50791272e+00,  4.16822104e+00, -2.13414308e+00, -2.66237999e-01,
        7.83819577e-01, -1.36703203e-01])]

In [ ]:
ieam6 = (0.004639595573920272, np.array([ 2.61411686e+00,  1.44689461e-02, -1.90339171e+02,  1.60713127e+02,
       -1.50065201e+02,  3.41733929e+01, -9.38215096e+00, -2.75667517e-01,
       -3.45258425e-01,  3.18322256e-01, -2.84002515e-01, -6.97804133e-02]))

In [ ]:
ieam9 = (0.008696488352891394, np.array([-1.47456565e+00, -4.79869485e-07, -1.68796012e+02,  1.01838014e+02,
       -1.89387663e+02,  1.23197119e+02,  1.67559787e+00, -1.97672693e+00,
        1.48460087e+00, -3.21160959e-01, -9.99990134e-01,  5.45243666e-01]))

In [ ]:
ieam6 = (0.012529698209073237, np.array([ 1.77179483e+00,  1.05192096e-02, -2.93005591e+02,  3.18328136e+02,
       -3.22036226e+02,  9.25544422e+01, -1.89150669e+01,  9.60458059e-01,
        9.09101177e-02, -2.93381125e-01,  1.16059169e-01, -1.21580547e-01]))

In [ ]:
ieam9 = (0.011166762673644966, np.array([-1.43897524e+00,  3.68357996e-06, -4.10835857e+01,  1.01633094e+02,
       -9.73045925e+01,  3.38437384e+01, -3.58251075e+00, -1.67919252e+00,
        8.65702805e-01,  2.65888291e-01, -1.20777286e+00,  4.75855652e-01]))

In [ ]:
ieam6 = (0.004926680089210157, np.array([-6.25140612e-01,  5.84924686e-05,  1.50463356e+01, -2.45628080e+00,
        2.75467959e+00, -1.82521768e+00,  5.82623873e-01, -8.10349686e-02]))

In [ ]:
(0.004856077250484402, array([-6.38942671e-01,  3.05962063e-05,  1.49946951e+01, -2.51929092e+00,
        2.70968892e+00, -1.67452072e+00,  5.32707246e-01, -6.59031531e-02,
        3.26582672e+00]))

In [ ]:
optimal_parameters = [ieam6]

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['relax']['xyz']])[:-2]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, targets, utot_EAM_per_box, [5])
    plt.title('Selected configurations at 0K')
    if i == 0:
        print('Targ:', targ_out[1][:-2])
        print('Model:', opti_out[1][:-2])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[1][:-2]))/nat)
        plt.plot(np.array(targ_out[1][:-2])/nat, 'o', label='target'+str(i), markersize=10)
        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    print('Model:', opti_out[1][:-2])

    #if i != 10 and i != 12:
    #    continue
    print(i, params_uopt[0])

        
    #print('Opti:', [o for o in opti_out[-1][:-1]])
    plt.plot(np.array([o for o in opti_out[1][:-2]])/nat, 'x--', label='optimized '+str(i))
#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]')
#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.7533)
#plt.ylim(-8.9, -8.35)
#plt.xlim(0, 4)

plt.legend()

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['eos_bcc']['xyz']])[:-1]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, targets, utot_EAM_per_box, [5])
    plt.title('Selected configurations at 0K')
    if i == 0:
        print('Targ:', targ_out[2][:])
        print('Model:', opti_out[2][:])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[2][:-1]))/nat)
        plt.plot(np.array(targ_out[2][:-1])/nat, 'o', label='target'+str(i), markersize=10)
        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    #if i != 10 and i != 12:
    #    continue
    print(i, params_uopt[0])

        
    #print('Opti:', [o for o in opti_out[-1][:-1]])
    plt.plot(np.array([o for o in opti_out[2][:-1]])/nat, 'x--', label='optimized '+str(i))
#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

#plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]')
#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.7533)
#plt.ylim(-8.9, -8.35)
#plt.xlim(0, 4)

plt.legend()

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['eos_fcc']['xyz']])[:-1]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, targets, utot_EAM_per_box, [5])
    plt.title('Selected configurations at 0K')
    if i == 0:
        print('Targ:', targ_out[-3][:])
        print('Model:', opti_out[-3][:])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[3][:-1]))/nat)
        plt.plot(np.array(targ_out[3][:-1])[[2, 3, 0, 1, 4, 5, 6, 7]]/nat, 'o', label='target'+str(i), markersize=10)
        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    #if i != 10 and i != 12:
    #    continue
    print(i, params_uopt[0])

        
    #print('Opti:', [o for o in opti_out[-1][:-1]])
    plt.plot(np.array([o for o in opti_out[3][:-1]])[[2, 3, 0, 1, 4, 5, 6, 7]]/nat, 'x--', label='optimized '+str(i))
#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

#plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]')
#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.7533)
#plt.ylim(-8.9, -8.35)
#plt.xlim(0, 4)

plt.legend()

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['eos_fcc']['xyz']])[:-1]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, targets, utot_EAM, [9])
    plt.title('Selected configurations at 0K')
    if i == 0:
        print('Targ:', targ_out[-1][:])
        print('Model:', opti_out[-1][:])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[-1][:-1]))/nat)
        plt.plot(np.array(targ_out[-1][:-1])[[2, 3, 0, 1, 4, 5, 6, 7, 8]]/nat, 'o', label='target'+str(i), markersize=10)
        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    #if i != 10 and i != 12:
    #    continue
    print(i, params_uopt[0])

        
    #print('Opti:', [o for o in opti_out[-1][:-1]])
    plt.plot( np.array([o for o in opti_out[-1][:-1]])[[2, 3, 0, 1, 4, 5, 6, 7, 8]]/nat, 'x--', label='optimized '+str(i))
#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

#plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]')
#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.7533)
#plt.ylim(-8.9, -8.35)
#plt.xlim(0, 4)

plt.legend()

In [ ]:
plt.figure(figsize=(20,10))

nat = np.array([len(x) for x in targ_ready['md']['xyz']])[:]
print(len(nat))

for i, params_uopt in enumerate(optimal_parameters):

    #print(params_uopt[1])

    opti_out, targ_out = udif_print(params_uopt[1], stats, targets, utot_EAM_per_box,[5])
    #plt.title('Selected configurations at 0K')
    if i == 0:
        #print('Targ:', targ_out[0][:])
        #print('Model:', opti_out[0][:])

        #nat = np.array([len(x) for x in targ_dict['relax']['xyz']])
        print(len(np.array(targ_out[0][:]))/nat)
        #plt.plot(np.array(targ_out[0][:])/nat, 'o', label='target'+str(i), markersize=10)
        plt.plot(np.array(targ_out[0][:]), 'o', label='target'+str(i), markersize=5)

        
    #if params_uopt[0] > 0.6e-3:
    #    continue
    #if i != 1:# and i != 12:
    #    continue
    print(i, params_uopt[0])

    #print('Opti:', [o for o in opti_out[-1][:-1]])
    #plt.plot(np.array([o for o in opti_out[0][:]])/nat, 'x--', label='optimized '+str(i))
    plt.plot(np.array([o for o in opti_out[0][:]]), '--', label='optimized '+str(i), lw=1.0)

#    else:
#        plt.plot([o for o in opti_out[-1][:-1]], 'x--', label='optimized '+str(i))
#        plt.plot(targ_out[-1][:-1], 'o', markersize=10)

#plt.xticks([0, 1, 2, 3, 4, 5, 6], ['BCC','FCC', 'vacancy','110','111','liq','inf'])
plt.ylabel('Energy [eV]', fontsize=30)
plt.xlabel('configuation #', fontsize=30)
plt.xticks(fontsize=20)
plt.yticks(fontsize=20)

#plt.ylim(-1145, -1136)
#plt.ylim(-8.905, -8.533)
#plt.ylim(-8.9, -8.35)

#plt.ylim(-1075,-1035)
#plt.ylim(-1000,-900)
#plt.ylim(-950,-900)
#plt.ylim(-1136,-1133)
plt.ylim(-1180,-800)
plt.xlim(0,820)
#plt.xlim(847,990)

#plt.ylim(-1180,-800)

plt.legend(fontsize=30)
#plt.savefig(plt.savefig(os.path.join(working, 'bcc_300.png')))

In [ ]:
for i, params_uopt in enumerate(optimal_parameters):
    if params_uopt[0] > 1.1e-3:
        continue
    print(i, params_uopt[0], '\n', params_uopt[1])

In [ ]:
stats_opts[it]['hyperparams']

In [ ]:
pars_out = {}
pars_out['pair_nodes'] = stats_opts[it]['hyperparams']
pars_out['mb_node'] = 9
pars_out['params'] = [optimal_parameters[0]]

In [ ]:
optimal_list = []

#with open(os.path.join(working, 'pars_out.pickle'), 'rb') as fi:
#    optimal_list = pickle.load(fi)

#if type(optimal_list) != list:
#    optimal_list = [optimal_list]

optimal_list.append(pars_out)

with open(os.path.join(working, 'pars_out.pickle'), 'wb') as fo:
    pickle.dump(optimal_list, fo)

In [ ]:
with open(os.path.join(working, 'pars_out.pickle'), 'rb') as fi:
    optimal_list = pickle.load(fi)

In [ ]:
optimal_list

In [ ]:
# Total pair potential
pos = targ_ready['relax']['xyz'][0]
bx = targ_ready['relax']['box'][0]
rr, rx = pair_dist_cutoff(pos, bx, 5.5)

In [ ]:
# embedding function
F = lambda d, a: a[0]*d**0.5 + a[1]*d + a[2]*d**2

V = lambda r, aa, cc: sum([a*(rc - r)**3 for a, rc in zip(aa, cc) if r < rc])

# Cohesive potential (cubic splines - same form as V)
phi = lambda r, aa, cc: sum([a*(rc - r)**3 for a, rc in zip(aa, cc) if r < rc])

rho = lambda r, aa, kk: sum([a*(rk - r)**3 for a, rk in zip(aa, kk) if r < rk and r > 0.01])

In [ ]:
optimal_list[-1]['params'][0][1]

In [ ]:
#optimal_parameters.append(tuple([output[1:], params_uopt]))
pair_y = optimal_list[-1]['params'][0][1][2:]
par = optimal_list[-1]['params'][0][1][:2]
many_y = [par[0], 0.0, par[1]]
V_x_r = optimal_list[-1]['pair_nodes'] #stats_opt['hyperparams']
rho_x_r = [V_x_r[optimal_list[-1]['mb_node']]] #stats_opt['hyperparams'][4:5]
rho_x_a = [1.0]

In [ ]:
pair_y, many_y, V_x_r, rho_x_r

In [ ]:
ene = []
for a in np.linspace(0.5, 2.0, 101):
    z = a*rr
    u_pair = 0.5*np.sum([V(x, pair_y, V_x_r) for x in z.flatten() if x > 0.1])

    u_many = 0.0
    for i in range(z.shape[0]):
        d = np.sum([phi(x, rho_x_a, rho_x_r) for x in z[i] if x > 0.1])
        u_many += F(d, many_y)

    ene.append([a, u_pair, u_many, u_pair+u_many])

ene = np.array(ene)

In [ ]:
#plt.plot(ene[:,0], ene[:,1]/54, label='pair')
#plt.plot(ene[:,0], ene[:,2]/54, label='manybody')
plt.title('Equation of state (T=0K)')
plt.plot(ene[:,0]*2.74, [universal_eos(x, 'W') for x in ene[:,0]], label='universal EOS')
plt.plot(ene[:,0]*2.74, ene[:,3]/128, label='u-total')
plt.hlines(0.0,0.5,6.0,lw=1)
plt.legend()
plt.xlim(2.0, 5.75)
plt.xlim(2.6, 2.85)
plt.ylim(-10,10)
plt.ylim(-9.,-8.5)

plt.xlabel('x')
plt.ylabel(r'U')

In [ ]:
#optimal_parameters.append(tuple([output[1:], params_uopt]))
pair_y = optimal_list[-1]['params'][0][1][2:]
par = optimal_list[-1]['params'][0][1][:2]
many_y = [par[0], 0.0, par[1]]
V_x_r = optimal_list[-1]['pair_nodes'] #stats_opt['hyperparams']
rho_x_r = [V_x_r[optimal_list[-1]['mb_node']]] #stats_opt['hyperparams'][4:5]
rho_x_a = [1.0]

In [ ]:
pair_y, many_y

In [ ]:
print(V_x_r)
print((0.011166762673644966, np.array([-1.43897524e+00,  3.68357996e-06, -4.10835857e+01,  1.01633094e+02,
       -9.73045925e+01,  3.38437384e+01, -3.58251075e+00, -1.67919252e+00,
        8.65702805e-01,  2.65888291e-01, -1.20777286e+00,  4.75855652e-01])))

In [ ]:
pair_a = optimal_list[-1]['params'][0][1][2:]#pars_in1[2:]
pair_r = optimal_list[-1]['pair_nodes']

rho_a = [1.0]
rho_r = [pair_r[optimal_list[-1]['mb_node']]]

par = optimal_list[-1]['params'][0][1][:2]
many_a = [par[0], 0.0, par[1]]

In [ ]:
pair_a, pair_r, rho_r, many_a, par

In [ ]:
# Ranges of densities and separations

nr = 5000
rcut = pair_r[-1]#stats_opt['hyperparams'][-1]
dr = rcut/nr

rhomax = 12.0*phi(1.5, rho_a, rho_r)
print('rhomax:', rhomax)
rhomax = 1000.0
nrho = 5000
drho = rhomax/nrho

ri = 1.0
ro = 2.0

# W properties
na = 74
ma = 183.84
lc = 3.1653
alat = 'bcc'

# System composition
n_el = 1
el_name = ['W']

In [ ]:
def u_trans(r, unucl, ueq, ri=1.0, ro=2.0):
    """Transition between the core and equillibrium potentials"""
    if r < ri:
        return unucl
    elif r < ro:
        x = (ro + ri - 2*r)/(ro - ri)
        eta = 3/16*x**5 - 5/8*x**3 + 15/16*x + 1/2
        return ueq + eta*(unucl - ueq)
    else:
        return ueq

In [ ]:
# prepare lists of values
dens = np.linspace(0., rhomax-drho, nrho, endpoint=True)
r = np.linspace(0., rcut-dr, nr, endpoint=True)
r2 = np.linspace(0., rcut, 201)

fembd = [F(d, many_a) for d in dens] # W
edens = np.array([phi(x, rho_a, rho_r) for x in r])

# 1. Cubic spline for r > r_o
f_outer = [V(x, pair_a, pair_r) for i, x in enumerate(r)]
# 2. Repulsive core for r < r_i (precalculate up to r_o)
f_inner = [0.0] + [u_core(x) for x in r[1:]]
# 3. Transition region for r_i < r < r_o
fpair = [x*u_trans(x, f_inner[i], f_outer[i]) for i, x in enumerate(r)]
#fpair = f_outer

In [ ]:
# LAMMPS format
pot_path = '../sim/potentials'

with open(os.path.join(pot_path, 'W_lv_bccfcc.eam.fs'), 'w') as fo:
    
    # Comment section
    fo.write('Comment 1\n')
    fo.write('Comment 2\n')
    fo.write('Comment 3\n')
    
    # Number of elements
    fo.write(f"{n_el:5d} ")
    for i in range(n_el):
        fo.write(f"{el_name[i]:2}")
    fo.write("\n")
    
    # number of grid points, step size, and cutoff
    fo.write(f"{nrho:5d}{drho:24.16e}{nr:5d}{dr:24.16e}{rcut:24.16e}\n")
    
    # atomic number, mass, lattice size, lattice type
    fo.write(f"{na:5d} {ma:14.4f} {lc:14.4f} {alat:10}\n")
    
    # Embeding function
    for i in range(nrho//4):
        fo.write("{0:20.12e}{1:20.12e}{2:20.12e}{3:20.12e}\n".format(*fembd[i*4:i*4+4]))
    
    # Electronic density
    for i in range(nr//4):
        fo.write("{0:20.12e}{1:20.12e}{2:20.12e}{3:20.12e}\n".format(*edens[i*4:i*4+4]))
        
    # Pair potential
    for i in range(nr//4):
        fo.write("{0:20.12e}{1:20.12e}{2:20.12e}{3:20.12e}\n".format(*fpair[i*4:i*4+4]))